In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import json
import matplotlib.pyplot as plt
pd.options.display.max_rows = 330

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow import keras

from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [10]:
def get_data(participants_to_include=0, split='within', window=12):
        
    
    data = pd.read_pickle('Preprocessed_2021-03-15.pkl')

    pilot_participants = ['gv_m_26','js_m_27','mb_f_30','tt_m_23', 'ys_m_29.1']
    experiment_participants = ['al_m_23','bd_f_25','el_m_26','gh_m_29','lb_f_19',\
                               'mb_f_28','om_m_31','rl_f_24',\
                               'rt_f_21','sf_m_24','ts_f_26',]

    participants = pilot_participants
    participants+=experiment_participants

    eeg_cols = []
    eeg_cols += ["baf_"+str(x) for x in [*range(1,122)]]
    eeg_cols += ["tf_"+str(x) for x in [*range(1,104)]]
    eeg_cols += ["VC_"+str(x) for x in [*range(0,13)]]
    eeg_cols += ["A_"+str(x) for x in [*range(0,14)]]
    
#     participants
    df = data[data['participant']==participants[participants_to_include]]
#     df = data
#     X_a = np.array(data.loc[:,eeg_cols])
#     print(X_a.shape)


    
#     y_a = np.array(data['participant'])
#     print(y_a.shape)
    
    
    # The EEG data is a matrix of 1673 rows and 251 columns. Need to reshape to 144 x WINDOW x 251
    # then we can crop it down
    loaded = list()
    y = list()

    for a in df.uniqueTrialCounter.unique(): #runs from 0 to 143
        #load x element
        b = np.array(data.loc[data.uniqueTrialCounter == a,'baf_1':'A_13'])[:window,:]
        loaded.append(b)
        #load y element
        myindex = data[data['uniqueTrialCounter']==a].index[0]
        y.append(data.iloc[myindex]['label'])

    loaded = keras.preprocessing.sequence.pad_sequences(loaded, dtype="float",)
    loaded = np.stack(loaded)
    print(loaded.shape)
    X = loaded

    y = np.array(y)
    
    print(y.shape)
    y = y.reshape(len(y),1)
    y.shape
#     y = 
    
    print(2*y.shape[0]//3)
    
    # np.max(X)
#     normalizer = Normalization(axis=-1)
#     normalizer.adapt(X)
    
#     Xn = normalizer(X)
    
    trainX = X[0:2*y.shape[0]//3]/np.max(X)
    testX = X[2*y.shape[0]//3:]/np.max(X)
    trainy = y[0:2*y.shape[0]//3]
    testy = y[2*y.shape[0]//3:]

    
    mean = trainX.mean(axis=0)
    std = trainX.std(axis=0)
    trainX -= mean
    trainX /=std
    
    testX -= mean
    testX /=std
#     trainy=to_categorical(trainy)
#     testy=to_categorical(testy)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    

    print(trainX.shape, trainy.shape, testX.shape, trainy.shape)
    
    return(trainX, trainy, testX, testy)

In [11]:
def generateModel1(n_timesteps, n_features,n_outputs):
    model = Sequential()
    model.add(LSTM(100,  input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def generateModel2(n_timesteps, n_features,n_outputs):
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps ,n_features)))
# #     model.add(LSTM(100))
    model.add(Dropout(0.2))
#     model.add(LSTM(100))
#     model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [12]:

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [14]:
repeats=2
# repeat experiment
scores = list()
for r in range(repeats):
    # load data
    trainX, trainy, testX, testy = get_data(0, 'within', 9)
    
    verbose, epochs, batch_size = 1, 50, 20
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print(n_timesteps, n_features, n_outputs)
    model = generateModel1(n_timesteps ,n_features, n_outputs)
 
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, score = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    # summarize results
summarize_results(scores)



(144, 9, 251)
(144,)
96
(96, 9, 251) (96, 3) (48, 9, 251) (96, 3)
9 251 3
Epoch 1/50
5/5 [==============================] - 2s 6ms/step - loss: 1.1755 - accuracy: 0.2729
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 0.9813 - accuracy: 0.5604
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 0.9447 - accuracy: 0.6028
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.8624 - accuracy: 0.7306
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 0.7635 - accuracy: 0.7688
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.7176 - accuracy: 0.8049
Epoch 7/50
5/5 [==============================] - 0s 5ms/step - loss: 0.6276 - accuracy: 0.8694
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.5770 - accuracy: 0.8465
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.5648 - accuracy: 0.8257
Epoch 10/50
5/5 [==============================] - 0s 4ms/step

5/5 [==============================] - 0s 6ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 27/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 28/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0274 - accuracy: 0.9965
Epoch 29/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 30/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0181 - accuracy: 1.0000
Epoch 31/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0153 - accuracy: 1.0000
Epoch 32/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 33/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0172 - accuracy: 1.0000
Epoch 34/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 35/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0157 - accuracy: 1.0000
Epoch 36/50
5/5 [=========================

In [35]:
testX[0]
# model.predict((testX[0]))

<tf.Tensor: shape=(6, 251), dtype=float32, numpy=
array([[ 0.40519565,  0.26390797,  0.2655608 , ...,  0.119133  ,
         0.13291448,  0.41024023],
       [ 0.39556843,  0.26244253,  0.20430121, ..., -0.07544658,
         0.1272642 ,  0.3301033 ],
       [ 0.3535288 , -0.1323592 , -0.18463203, ...,  0.06584046,
        -0.00892712, -0.12109778],
       [ 0.21057819, -0.1419756 , -0.04721254, ...,  0.03034545,
        -0.02410885,  0.0777398 ],
       [-0.02604828, -0.18671046,  0.01773684, ..., -0.13395451,
        -0.08709727,  0.07572434],
       [ 0.06540973, -0.0554717 ,  0.1254056 , ..., -0.07783575,
        -0.10648037,  0.11190312]], dtype=float32)>

In [25]:
trainy

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1